<a href="https://colab.research.google.com/github/mohamad1371/RL_trader_bot/blob/main/MlpLstm_heiken_ashi_%26_RSI_5min_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-binance
!pip install stable_baselines
!pip install tensorflow==1.5

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, timezone

from binance.client import Client

client = Client()

def get_market_data(asset, timeFrame='1d', fromTime='1 Jan, 2019', saveFile=False):  
    """"""    
    
    tf = translate_time_frame(timeFrame)
    
#    try:  
#        
#    except:
#        return pd.DataFrame()
    
    candles = client.get_historical_klines(asset, tf, fromTime)
    arrData = np.array(candles)
    
    dates = arrData[:,0].reshape(-1,1)
    dates = [epoch_to_datetime(t) for t in dates]                    

    df = pd.DataFrame(index=dates)
    df.index.name = 'Date Time'
    df['Open'] = arrData[:,1]
    df['High'] = arrData[:,2]
    df['Low'] = arrData[:,3]
    df['Close'] = arrData[:,4]
    df['Volume'] = arrData[:,5]
    df['Adj Close'] = df['Close']
    df['Quote Volume'] = arrData[:,7]
    df['Trades Count'] = arrData[:,8]
    
    df = df[:-1]

    if saveFile:            
        fromTime = df.index[0]
        toTime = df.index[-1]
        fileName = 'Binance_{}_{}_{}_{}.csv'.format(asset, timeFrame, fromTime, toTime).replace(':', '-')
        df.to_csv(fileName, index=True)

    return df

# %%
    
def epoch_to_datetime(epoch):
    return datetime.fromtimestamp(int(epoch) // 1000, tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S')

# %%
  
def translate_time_frame(tf):

    mapping = {
        "1m": Client.KLINE_INTERVAL_1MINUTE,
        "3m": Client.KLINE_INTERVAL_3MINUTE,
        "5m": Client.KLINE_INTERVAL_5MINUTE,
        "15m": Client.KLINE_INTERVAL_15MINUTE,
        "30m": Client.KLINE_INTERVAL_30MINUTE,
        "1h": Client.KLINE_INTERVAL_1HOUR,
        "2h": Client.KLINE_INTERVAL_2HOUR,
        "4h": Client.KLINE_INTERVAL_4HOUR,
        "6h": Client.KLINE_INTERVAL_1HOUR,
        "8h": Client.KLINE_INTERVAL_8HOUR,
        "12h": Client.KLINE_INTERVAL_12HOUR,
        "1d": Client.KLINE_INTERVAL_1DAY,
        "3d": Client.KLINE_INTERVAL_3DAY,
        "1w": Client.KLINE_INTERVAL_1WEEK,
        "1M": Client.KLINE_INTERVAL_1MONTH
    }

    return mapping.get(str(tf))

asset='BTCUSDT'
df=get_market_data(asset, timeFrame='5m', fromTime='1 Jan, 2017', saveFile=True)

#KLINE_INTERVAL_1MINUTE = '1m'
#KLINE_INTERVAL_3MINUTE = '3m'
#KLINE_INTERVAL_5MINUTE = '5m'
#KLINE_INTERVAL_15MINUTE = '15m'
#KLINE_INTERVAL_30MINUTE = '30m'
#KLINE_INTERVAL_1HOUR = '1h'
#KLINE_INTERVAL_2HOUR = '2h'
#KLINE_INTERVAL_4HOUR = '4h'
#KLINE_INTERVAL_6HOUR = '6h'
#KLINE_INTERVAL_8HOUR = '8h'
#KLINE_INTERVAL_12HOUR = '12h'
#KLINE_INTERVAL_1DAY = '1d'
#KLINE_INTERVAL_3DAY = '3d'
#KLINE_INTERVAL_1WEEK = '1w'
#KLINE_INTERVAL_1MONTH = '1M'

In [ ]:
import pandas as pd

N_STEPS = 100000
df = pd.read_csv('content/Binance_BTCUSDT_5m_2017-08-17 04-00-00_2020-04-03 08-00-00.csv')
# df.rename(columns = {' High':'High', ' Low':'Low', 
#                               ' Close':'Close',' Volume':'Volume'}, inplace = True)
df=df.iloc[100000:200000]
# df=df.head(N_STEPS)


df.to_csv('data_5_min.csv')

In [ ]:
import math
import pandas as pd
df = pd.read_csv('content/data_5_min.csv')
op=[0]
hi=[0]
lo=[0]
cl=[0]
vl=[0]


HAclose = df[["Open", "High", "Low", "Close"]].sum(axis=1) / 4

for i in range(len(df['Close'])-1):
    
#     op.append((math.log10(df['Open'][i+1])-math.log10(df['Open'][i])))
#     hi.append((math.log10(df['High'][i+1])-math.log10(df['High'][i])))
#     lo.append((math.log10(df['Low'][i+1])-math.log10(df['Low'][i])))
    cl.append((math.log10(HAclose[i+1])-math.log10(HAclose[i])))


In [ ]:
import numpy as np



# diff_open= pd.DataFrame(np.array(op),columns=['Open'])

# diff_high= pd.DataFrame(np.array(hi),columns=['High'])

# diff_low= pd.DataFrame(np.array(lo), columns=['Low'])

diff_close = pd.DataFrame(np.array(cl), columns=['Close'])


In [ ]:
import random
import json
import gym
from gym import spaces
import pandas as pd
import numpy as np
from statistics import mean
import talib
INITIAL_CASH = 60000
LOOK_BACK=15


class StockTradingEnv(gym.Env):
    """A stock trading environment for OpenAI gym"""
    metadata = {'render.modes': ['human']}

    def __init__(self, df):
        super().__init__()

        self.df = df
        self.reward_range = (-200, 200)
        
        self.action_space=spaces.MultiDiscrete([3,2])
        self.p_val = []
        

        self.observation_space = spaces.Box(
            low=0.0, high=1.0, shape=(3,15), dtype=np.float64)

    def reset(self):
  
        self.cash = INITIAL_CASH
        self.portfolio_value = INITIAL_CASH
        self.rsi=50

        for i in range(LOOK_BACK):
            self.p_val.append(self.portfolio_value)
            
        self.transactions=[]        
        self.position_amount = 0
        self.trade_value=0
        self.current_step = LOOK_BACK
        return self._next_observation()

    def _next_observation(self):

        frame = np.array([
#             diff_open.loc[self.current_step-LOOK_BACK: self.current_step-1, 'Open'].values,
#             diff_high.loc[self.current_step-LOOK_BACK: self.current_step-1, 'High'].values,
#             diff_low.loc[self.current_step-LOOK_BACK: self.current_step-1, 'Low'].values,
            diff_close.loc[self.current_step-LOOK_BACK: self.current_step-1, 'Close'].values,
            self.df.loc[self.current_step-LOOK_BACK: self.current_step-1, 'Volume'].values,          
        ])

        self.cash = np.clip(float("{0:.5f}".format(self.cash)), 0, np.inf)
        self.portfolio_value = np.clip(float("{0:.5f}".format(self.portfolio_value)), 0, np.inf)
        self.position_amount = np.clip(float("{0:.5f}".format(self.position_amount)), 0, np.inf)
        rsii = talib.RSI(self.df.loc[self.current_step-LOOK_BACK: self.current_step-1, 'Close'].values, timeperiod=14)
        self.rsi = rsii[-1]

        obs = np.append(frame, [[
            self.cash,
            self.cash,
            self.cash,
            self.cash,
            self.portfolio_value,
            self.portfolio_value,
            self.portfolio_value,
            self.portfolio_value,
            self.position_amount,
            self.position_amount,
            self.position_amount,
            self.rsi,
            self.rsi,
            self.rsi,
            self.rsi
        ]], axis=0)
        return obs   

    def step(self, action):

        if len(self.p_val)> LOOK_BACK :
            self.p_val = self.p_val[-LOOK_BACK:]
        
        if len(self.transactions)> LOOK_BACK :
            self.transactions = self.transactions[-LOOK_BACK:]
            
        self._take_action(action)
        self.current_step += 1
        

        if self.current_step > len(self.df.loc[:, 'Open'].values)-(LOOK_BACK+1):
            self.current_step = LOOK_BACK 

        reward = self.get_reward(action)
        done = 1<0

        obs = self._next_observation()

        return obs, reward, done, {}   

    def _take_action(self, action):

        current_price = random.uniform(
            self.df.loc[self.current_step, "Open"], self.df.loc[self.current_step, "Close"])

        action_type = action[0]
        buy_amount = (action[1]+1)/2
        sell_amount = 1

        #buy
        if action_type == 0:
            
            self.cash, self.position_amount = self.buy(buy_amount, current_price)
 
        #sell    
        elif action_type == 1:
            self.cash, self.position_amount = self.sell(sell_amount, current_price)

        elif  action_type == 2:
              pass

        self.portfolio_value = self.cash + (self.position_amount * current_price)
        self.p_val.append(self.portfolio_value)
        # print(f'act: {action} p_val_act :{self.portfolio_value}')
    
    def commission(self, trade_value):

        fee = trade_value * 0.001
        return fee

    def sell(self, sell_amount, current_price):
        pos_amount_s = self.position_amount
        cash_s = self.cash
        shares_sold = (pos_amount_s * sell_amount)
        trade_value_s = shares_sold * current_price
        if trade_value_s > 0:
            self.transactions.append(trade_value_s)
            
        comm_s = self.commission(trade_value_s)
        trade_value_s -= comm_s
        cash_s += trade_value_s
        pos_amount_s -= shares_sold

        return cash_s, pos_amount_s

    def buy(self, buy_amount, current_price):
        cash_b = self.cash
        pos_amount_b = self.position_amount
        can_buy = cash_b / current_price
        can_buy = float("{0:.5f}".format(can_buy))
        shares_bought = can_buy * buy_amount
        trade_value_b = shares_bought * current_price
        if trade_value_b > 0:
            self.transactions.append(-trade_value_b)
        comm_b = self.commission(trade_value_b)
        # print(f"buying_val: {trade_value_b}\nfee: {comm_b}")   
        trade_value_b += comm_b

        cash_b -= trade_value_b
        pos_amount_b += shares_bought

        return cash_b, pos_amount_b

    def get_reward(self, action):

        if self.rsi<=30 and action[0]==0 :
            reward_= 10
        
        elif self.rsi>=70 and action[0]==1 and self.p_val[-1]>self.p_val[0] and self.transactions[-1]>0:
            reward_= 200
        
        elif self.rsi>=80 and action[0]==0:
            reward_= -200
        
        elif self.rsi<=20 and action[0]==1:
            reward_= -200
        
        elif self.rsi<70 and self.rsi>30 and action[0]==2:
            reward_= 20
        
        else:
            reward_= 0
            
        return reward_

    def render(self, mode='human', close=False):
        profit = self.portfolio_value - INITIAL_CASH
        cash = self.cash
        tot_held = self.position_amount
        portfo_val = self.portfolio_value

        return profit, cash, tot_held, portfo_val


In [ ]:
import json
import datetime as dt
from stable_baselines.common.policies import MlpLstmPolicy, MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv,SubprocVecEnv,VecNormalize
from stable_baselines import PPO2, A2C
import matplotlib.pyplot as plt
import pandas as pd

df = pd.read_csv('content/data_5_min.csv')

env = DummyVecEnv([lambda: StockTradingEnv(df)])

env=VecNormalize(env, training=True, norm_obs=True, norm_reward=True)

model = PPO2(MlpLstmPolicy, env, verbose=0, n_steps=8, gamma=0.9998, cliprange=0.08, ent_coef=0.02, nminibatches=1, noptepochs=4)
# model = A2C(MlpPolicy, env, verbose=1, n_steps=8, gamma=0.999, ent_coef=0.01)
model.set_env(env)

#this next 3 lines shuold be commented when using saved model

model.learn(total_timesteps = N_STEPS)
model.save('content/Stock-Trading-Env.pkl')
del model

model=PPO2.load('content/Stock-Trading-Env.pkl')

obs = env.reset()
state = None
done = [False for _ in range(env.num_envs)]

act=[]
p=[]
b=[]
th=[]
ts=[]
nw=[]
mn=[]
d=[]

for _ in range(N_STEPS):
    action, state = model.predict(obs, state=state, mask=done)
    obs, rewards , done, _ = env.step(action)
    profit, cash, tot_held, portfo_val = env.render()

    p.append(profit)
    b.append(cash)
    th.append(tot_held)
    nw.append(portfo_val)
    mn.append(rewards)
    act.append(action[0][0])

# # plt.plot(p, label='profit')
# plt.plot(b, label='cash')
# # plt.plot(act, label='action')
# # plt.plot(th[:100], label='tot_held')
# # plt.plot(ts, label='tot_sold')
# plt.plot(nw, label='portfolio value')
# # plt.plot(mn,label='rewards')

# plt.legend()
# plt.grid(linestyle='dotted')
# plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

a = pd.read_csv('content/data_5_min.csv')
a = a.iloc[LOOK_BACK : N_STEPS + LOOK_BACK]

# a=a.head(100000)
first_share=INITIAL_CASH/a['Close'][LOOK_BACK]
b_h=a['Close']*first_share


nw_prc=[]
ss=100/INITIAL_CASH
for i in nw:
    nw_prc.append((ss*(i-INITIAL_CASH)))

b_h_prc=[]
for j in b_h:
    b_h_prc.append((ss*(j-INITIAL_CASH)))


# plt.plot(b_h_prc, label='B&H %')
# plt.plot(nw_prc,label='portfolio value %')
# # plt.plot(act[:1000], label='action')
# # plt.plot(mn,label='rewards')
# # plt.plot(th, label='tot_held') 
# # plt.plot(nw, label='portfolio value')
# # plt.plot(b, label='cash')
# # # plt.plot(a['Volume'])
# plt.legend()
# plt.grid(linestyle='dotted')
# plt.show()

In [ ]:
import matplotlib.pyplot as plt

quote_currency = 'USD'
asset='BTC_CSD'
ax1 = plt.subplot(511)
nw = pd.DataFrame(np.array(nw), columns=['Port_val'])
nw.plot(ax=ax1)
ax1.set_ylabel('Portfolio\nValue\n({})'.format(quote_currency))

# Plot the price increase or decrease over time.
ax2 = plt.subplot(512, sharex=ax1)
df['Close'].plot(ax=ax2, color='silver', label='price')

ax2.set_ylabel('{}\n({})'.format(
    asset, quote_currency 
))

ax3 = plt.subplot(513, sharex=ax1)
th = pd.DataFrame(np.array(th), columns=['Total BTC held'])
th.plot(ax=ax3)
ax3.set_ylabel('Position_amount')

act = pd.DataFrame(np.array(act))
buy_df = act[act.values==0]

sell_df = act[act.values==1]

ax2.scatter(
    buy_df.index,
    df['Close'][buy_df.index],
    marker='^',
    s=100,
    c='red',
    label=''
)
ax2.scatter(
    sell_df.index,
    df['Close'][sell_df.index],
    marker='v',
    s=130,
    c='green',
    label=''
)

ax4 = plt.subplot(514, sharex=ax1)
b = pd.DataFrame(np.array(b), columns=['cash'])
b.plot(
    ax=ax4, label='Quote Currency ({})'.format(quote_currency)
)
ax4.set_ylabel('Cash\n({})'.format(quote_currency))


ax5 = plt.subplot(515, sharex=ax1)
b_h_prc=pd.DataFrame(np.array(b_h_prc), columns=['Price'])
b_h_prc.plot(ax=ax5)
nw_prc=pd.DataFrame(np.array(nw_prc), columns=['algorithm'])
nw_prc.plot(ax=ax5)
ax5.set_ylabel('Percent\nChange')
plt.legend(loc=3)
plt.gcf().set_size_inches(18, 38)
plt.show()